In [ ]:
#|default_exp cli

In [ ]:
#|export
from execnb.nbio import read_nb
from nbdev.processors import NBProcessor
from nbdev.export import ExportModuleProc, nb_export
from nbdev.maker import ModuleMaker
from fastcore.xtras import globtastic, Path
from functools import partial
from fastcore.script import call_parse
from nbdev import nbdev_export

/Users/deven367/miniforge3/envs/nixtla/lib/python3.11/site-packages/nbdev/doclinks.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources,importlib


In [ ]:
nb_path = "../nbs/evaluation.ipynb"
nb = read_nb(nb_path)

In [ ]:
#|export
tst_flags = 'datasets distributed matplotlib polars pyarrow scipy'.split()
to_skip = [
    'showdoc',
    'load_ext',
    'from nbdev'
]


def print_execs(cell):
    if 'exec' in cell.source: print(cell.source)

def print_hide(cell):
    if 'hide' in cell.directives_: print(cell.source)

def other_tests(cell):
    if len(cell.directives_) == 0:
        print(cell.source)

def get_markdown(cell):
    if cell.cell_type == "markdown":
        print(cell.source)

def extract_dir(cell, dir):
    if dir in cell.directives_:
        print(cell.source)

def no_dir_and_dir(cell, dir):
    if len(cell.directives_) == 0:
        print(cell.source)

    if dir in cell.directives_:
        print(cell.source)

def get_all_tests2(cell):
    if cell.cell_type == "code":

        if len(cell.directives_) == 0:
            print(cell.source)


        elif any(x in tst_flags + ['hide'] for x in cell.directives_):
            if not (x in cell.source for x in to_skip):
                print(cell.source)

def get_all_tests(cell):
    if len(cell.directives_) == 0:
        print(cell.source)

    if any(x in tst_flags + ["hide"] for x in cell.directives_):
        print(cell.source)



In [ ]:
tst_cell = nb.cells[0]

In [ ]:
tst_cell

```json
{ 'cell_type': 'code',
  'execution_count': None,
  'id': '5aace6e9-4c24-4e66-b786-f468e32227a6',
  'idx_': 0,
  'metadata': {},
  'outputs': [],
  'source': '#| hide\n%load_ext autoreload\n%autoreload 2'}
```

In [ ]:
tst_flags + ['hide']

['datasets', 'distributed', 'matplotlib', 'polars', 'pyarrow', 'scipy', 'hide']

In [ ]:
#|export
mapper = {
    'print_execs': print_execs,
    'print_hide': print_hide,
    'other_tests': other_tests,
    'get_markdown': get_markdown,
    'extract_dir': extract_dir,
    'no_dir_and_dir': no_dir_and_dir,
    'get_all_tests':get_all_tests
}

In [ ]:
#|export
@call_parse
def print_dir_in_nb(nb_path:str,
                    dir:str=None,
                    dir_name:str=None,
                    ):
    if dir_name not in mapper.keys():
        raise ValueError(f'Choose processor from the the following: {mapper.keys()}')

    if dir_name == 'extract_dir':
        processor = NBProcessor(nb_path, partial(extract_dir, dir=dir))
        processor.process()
        return
    elif dir_name == 'no_dir_and_dir':
        processor = NBProcessor(nb_path, partial(no_dir_and_dir, dir=dir))
        processor.process()
        return

    processor = NBProcessor(nb_path, mapper[dir_name])
    processor.process()


In [ ]:
NBProcessor(nb_path, procs=get_all_tests).process()

%load_ext autoreload
%autoreload 2
from nbdev import show_doc
show_doc(evaluate)
from functools import partial

import numpy as np
import pandas as pd

from utilsforecast.losses import *
from utilsforecast.data import generate_series
series = generate_series(10, n_models=2, level=[80, 95])
series['unique_id'] = series['unique_id'].astype('int')
models = ['model0', 'model1']
metrics = [
    mae,
    mse,
    rmse,
    mape,
    smape,
    partial(mase, seasonality=7),
    quantile_loss,
    mqloss,
    coverage,
    calibration,
    scaled_crps,
]
evaluation = evaluate(
    series,
    metrics=metrics,
    models=models,
    train_df=series,
    level=[80, 95],
)
evaluation
summary = evaluation.drop(columns='unique_id').groupby('metric').mean().reset_index()
summary
import polars.testing
series_pl = generate_series(10, n_models=2, level=[80, 95], engine='polars')
pl_evaluation = (
    evaluate(
        series_pl,
        metrics=metrics,
        train_df=series_pl,
        level=[80, 95

In [ ]:
processor = NBProcessor(nb_path, partial(extract_dir, dir='distributed'))

In [ ]:
processor.process()

import sys
from itertools import product

import dask.dataframe as dd
import fugue.api as fa
from pyspark.sql import SparkSession
if sys.version_info >= (3, 9):
    spark = SparkSession.builder.getOrCreate()
    spark.sparkContext.setLogLevel('FATAL')
    dask_df = dd.from_pandas(series, npartitions=2)
    spark_df = spark.createDataFrame(series).repartition(2)
    for distributed_df, use_train in product([dask_df, spark_df], [True, False]):
        distr_metrics = [rmse, mae]
        if use_train:
            distr_metrics.append(partial(mase, seasonality=7))
            local_train = series
            distr_train = distributed_df
        else:
            local_train = None
            distr_train = None
        local_res = evaluate(series, metrics=distr_metrics, level=level, train_df=local_train)
        distr_res = fa.as_fugue_df(
            evaluate(distributed_df, metrics=distr_metrics, level=level, train_df=distr_train)
        ).as_pandas()
        pd.testing.assert_frame_equ

In [ ]:
nb_export(nb_path, '../tests', partial(extract_dir, dir='distributed'), 'foo2')

In [ ]:
nbs = globtastic('../nbs', file_glob='*.ipynb', recursive=False).map(Path).sorted()
tst_flags = 'datasets distributed matplotlib polars pyarrow scipy'.split()

In [ ]:
nbs

(#12) [Path('../nbs/compat.ipynb'),Path('../nbs/data.ipynb'),Path('../nbs/evaluation.ipynb'),Path('../nbs/feature_engineering.ipynb'),Path('../nbs/grouped_array.ipynb'),Path('../nbs/index.ipynb'),Path('../nbs/losses.ipynb'),Path('../nbs/plotting.ipynb'),Path('../nbs/preprocessing.ipynb'),Path('../nbs/processing.ipynb'),Path('../nbs/read.ipynb'),Path('../nbs/validation.ipynb')]

In [ ]:
TEST_PATH = Path('../tests').resolve()

In [ ]:
TEST_PATH

Path('/Users/deven367/projects/public/utilsforecast/tests')

In [ ]:
if not TEST_PATH.exists():
    TEST_PATH.mkdir()

In [ ]:
nbs[4]

Path('../nbs/grouped_array.ipynb')

In [ ]:
# for tst in tst_flags:
#     for nb in nbs:
#         nb_name = nb.stem
#         nb_export(nb, lib_path='../tests', procs=partial(extract_dir, dir=tst), name=f'{tst}_{nb_name}')

In [ ]:
# for tst in tst_flags:
#     for nb in nbs:
#         nb_name = nb.stem
#         code = NBProcessor(nb, partial(extract_dir, dir=tst))
#         with open(TEST_PATH / f'{tst}.py', '+a') as f:
#             if code.process() is not None:
#                 f.write(code.process())